In [1]:
!pip install -q plotly
!pip install kaleido==0.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 21.0 MB/s eta 0:00:00:00:0100:01


In [2]:
# Requirements:
# pip install yfinance tensorflow pandas numpy scikit-learn matplotlib tqdm

import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tqdm import tqdm
from tensorflow.keras.models import Sequential
import os
import plotly.graph_objects as go
import kaleido
import plotly.io as pio
pio.kaleido.scope.default_format = "png"

import warnings
warnings.filterwarnings('ignore')

2025-11-18 10:10:32.015836: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763460632.423045      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763460632.533745      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
STOCK_TO_SECTOR = {
    # Auto
    "BAJAJ-AUTO": "Auto", "EICHERMOT": "Auto", "M&M": "Auto",
    "MARUTI": "Auto", "TATAMOTORS": "Auto",

    # Bank
    "AXISBANK": "Bank", "HDFCBANK": "Bank", "ICICIBANK": "Bank",
    "KOTAKBANK": "Bank", "SBIN": "Bank",

    # Chemicals
    "PIDILITIND": "Chemicals", "PIIND": "Chemicals", "SOLARINDS": "Chemicals",
    "SRF": "Chemicals", "UPL": "Chemicals",

    # FMCG
    "BRITANNIA": "FMCG", "HINDUNILVR": "FMCG", "ITC": "FMCG",
    "NESTLEIND": "FMCG", "TATACONSUM": "FMCG",

    # Healthcare
    "APOLLOHOSP": "Healthcare", "CIPLA": "Healthcare", "DIVISLAB": "Healthcare",
    "MAXHEALTH": "Healthcare", "SUNPHARMA": "Healthcare",

    # IT
    "HCLTECH": "IT", "INFY": "IT", "TCS": "IT", "TECHM": "IT", "WIPRO": "IT",

    # Media
    "NAZARA": "Media", "PVRINOX": "Media", "SAREGAMA": "Media",
    "SUNTV": "Media", "ZEEL": "Media",

    # Metal
    "ADANIENT": "Metal", "HINDALCO": "Metal", "JSWSTEEL": "Metal",
    "TATASTEEL": "Metal", "VEDL": "Metal",

    # Oil & Gas
    "BPCL": "OilGas", "GAIL": "OilGas", "IOC": "OilGas",
    "ONGC": "OilGas", "RELIANCE": "OilGas",

    # Pharma
    "CIPLA": "Pharma", "DIVISLAB": "Pharma", "DRREDDY": "Pharma",
    "LUPIN": "Pharma", "SUNPHARMA": "Pharma"
}

In [ ]:
#24th nov used for model summary saving
# ------------------------------
# GPU (optional)
# ------------------------------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU(s) memory growth enabled:", gpus)
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found, using CPU")

# ------------------------------
# Config (user)
# ------------------------------

ticker = "RELIANCE.NS"
end_date = datetime.today()
start_date = end_date - timedelta(days=730)
window = 10
step = 5
epochs = 30
batch_size = 16
forecast_days = 5
random_seed = 42
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

# ------------------------------
# 1. Load data (trading days only)
# ------------------------------
df = yf.download(
    ticker,
    start=start_date.strftime('%Y-%m-%d'),
    end=end_date.strftime('%Y-%m-%d'),
    progress=False
)

if df.empty:
    raise RuntimeError("No data downloaded - check the ticker or date range.")

df = df[['Close']].dropna()
prices = df['Close'].values.reshape(-1, 1)
dates = pd.to_datetime(df.index)

# ------------------------------
# 2. Helper: create supervised data (returns X, y)
# ------------------------------
def create_dataset(arr, window=10):
    X, y = [], []
    for i in range(len(arr) - window):
        X.append(arr[i:i+window])
        y.append(arr[i+window])
    return np.array(X), np.array(y)

# ------------------------------
# 3. Directional accuracy
# ------------------------------
def directional_accuracy(actual, predicted):
    actual = np.array(actual).flatten()
    predicted = np.array(predicted).flatten()
    if len(actual) < 2 or len(predicted) < 2:
        return np.nan
    actual_move = actual[1:] - actual[:-1]
    pred_move = predicted[1:] - predicted[:-1]
    actual_dir = (actual_move > 0).astype(int)
    pred_dir = (pred_move > 0).astype(int)
    correct = (actual_dir == pred_dir).sum()
    total = len(actual_dir)
    return correct / total * 100

# ------------------------------
# 4. Model factories
# ------------------------------
def build_lstm(window):
    inp = Input(shape=(window, 1))
    x = LSTM(64, return_sequences=True)(inp)
    x = Dropout(0.2)(x)
    x = LSTM(32)(x)
    x = Dense(16, activation='relu')(x)
    out = Dense(1)(x)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.Huber(delta=1))
    return model

def build_cnn(window):
    model = Sequential([
        Conv1D(32, 2, activation='relu', padding='same', input_shape=(window,1)),
        Dropout(0.2),
        MaxPooling1D(pool_size=2, padding='same'),
        Flatten(),
        Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.Huber(delta=1))
    return model

def build_hybrid(window):
    inp = Input(shape=(window,1))
    x = Conv1D(32, 2, activation='relu', padding='same')(inp)
    x = Dropout(0.2)(x)
    x = LSTM(32, kernel_regularizer=l2(0.001))(x)
    x = Dense(1)(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.Huber(delta=1))
    return model

# ------------------------------
# 5. Walk-forward evaluation
# ------------------------------
def walk_forward_eval(prices, dates, model_type='lstm', window=10, step=5,
                      epochs=30, batch_size=16):

    preds_all, actuals_all, dates_all = [], [], []
    es = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)

    for i in tqdm(range(window, len(prices) - step + 1, step), desc=f"WF: {model_type}"):
        train = prices[:i].copy()
        test = prices[i:i+step].copy()
        train_dates = dates[:i]
        test_dates = dates[i:i+step]

        scaler = StandardScaler()
        train_scaled = scaler.fit_transform(train)
        test_scaled = scaler.transform(test)

        X_train, y_train = create_dataset(train_scaled, window)

        concat = np.vstack([train_scaled[-window:], test_scaled])
        X_test, y_test = create_dataset(concat, window)

        if X_train.size == 0 or X_test.size == 0:
            continue

        X_train = X_train.reshape((X_train.shape[0], window, 1))
        X_test = X_test.reshape((X_test.shape[0], window, 1))

        model = (
            build_lstm(window) if model_type == "lstm"
            else build_cnn(window) if model_type == "cnn"
            else build_hybrid(window)
        )

        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                  verbose=0, callbacks=[es])

        pred_scaled = model.predict(X_test, verbose=0).flatten()
        pred_inv = scaler.inverse_transform(pred_scaled.reshape(-1,1)).flatten()
        actual_inv = scaler.inverse_transform(y_test.reshape(-1,1)).flatten()

        preds_all.extend(pred_inv.tolist())
        actuals_all.extend(actual_inv.tolist())
        dates_all.extend(list(test_dates))

    preds_arr = np.array(preds_all)
    actual_arr = np.array(actuals_all)

    mse = mean_squared_error(actual_arr, preds_arr)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual_arr, preds_arr)
    rmse_mean = rmse / np.mean(actual_arr)
    da = directional_accuracy(actual_arr, preds_arr)

    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'rmse_mean_ratio': rmse_mean,
        'directional_acc': da,
        'preds': preds_arr,
        'actuals': actual_arr,
        'dates': pd.to_datetime(dates_all)
    }

# ------------------------------
# 6. Run walk-forward
# ------------------------------
models = ['cnn', 'lstm', 'hybrid']
results = {}
all_preds_df = None

for m in models:
    res = walk_forward_eval(prices, dates, m, window, step, epochs, batch_size)
    results[m] = {
        'MSE': res['mse'],
        'RMSE': res['rmse'],
        'MAE': res['mae'],
        'RMSE/MEAN': res['rmse_mean_ratio'],
        'DirectionalAcc': res['directional_acc']
    }

    df_m = pd.DataFrame({
        'Date': res['dates'],
        f'actual_{m}': res['actuals'],
        f'pred_{m}': res['preds']
    })

    all_preds_df = df_m if all_preds_df is None else pd.merge(
        all_preds_df, df_m, on='Date', how='outer'
    )

all_preds_df = all_preds_df.sort_values('Date').reset_index(drop=True)

actual_cols = [c for c in all_preds_df.columns if c.startswith('actual_')]
if actual_cols:
    all_preds_df['actual'] = all_preds_df[actual_cols].bfill(axis=1).iloc[:, 0]
    all_preds_df.drop(columns=actual_cols, inplace=True)

all_preds_df = all_preds_df[['Date', 'actual'] + [c for c in all_preds_df.columns if c.startswith("pred_")]]

# ------------------------------
# 7. Retrain full model + forecast
# ------------------------------
def retrain_and_forecast(prices, dates, model_type='lstm', window=10, forecast_days=5,
                         epochs=30, batch_size=16):

    scaler = StandardScaler()
    scaled = scaler.fit_transform(prices)

    X, y = create_dataset(scaled, window)
    X = X.reshape((X.shape[0], window, 1))

    model = (
        build_lstm(window) if model_type == "lstm"
        else build_cnn(window) if model_type == "cnn"
        else build_hybrid(window)
    )

    es = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[es])

    last = scaled[-window:].reshape(1, window, 1)
    preds_scaled = []
    temp = last.copy()

    for _ in range(forecast_days):
        f = model.predict(temp, verbose=0)
        preds_scaled.append(float(f.flatten()[0]))
        temp = np.append(temp[:,1:,:], f.reshape(1,1,1), axis=1)

    preds = scaler.inverse_transform(np.array(preds_scaled).reshape(-1,1)).flatten()
    future_dates = pd.date_range(start=dates[-1], periods=forecast_days+1, freq='B')[1:]

    return pd.DataFrame({'Date': future_dates, f'{model_type}_forecast': preds}), model

forecast_dates = pd.date_range(start=dates[-1], periods=forecast_days+1, freq='B')[1:]
forecast_df = pd.DataFrame({'Date': forecast_dates})
retrained_models = {}

for m in models:
    fc, mdl = retrain_and_forecast(prices, dates, m, window, forecast_days, epochs, batch_size)
    forecast_df = pd.merge(forecast_df, fc, on='Date', how='left')
    retrained_models[m] = mdl

# ------------------------------
# 8. Build metrics dataframe (with requested columns)
# ------------------------------
def compute_mape(a, p):
    a = np.array(a).astype(float)
    p = np.array(p).astype(float)
    mask = a != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((a[mask] - p[mask]) / a[mask])) * 100

metrics_rows = []
for m in models:
    pred_col = f'pred_{m}'
    if pred_col not in all_preds_df:
        continue

    preds = all_preds_df[pred_col].dropna().values
    actual = all_preds_df['actual'].iloc[:len(preds)].values

    if len(preds) == 0:
        mse = rmse = mae = rmse_mean = da = mape = np.nan
    else:
        mse = mean_squared_error(actual, preds)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actual, preds)
        rmse_mean = rmse / np.mean(actual) if np.mean(actual) != 0 else np.nan
        da = directional_accuracy(actual, preds)
        mape = compute_mape(actual, preds)

    # You had a STOCK_TO_SECTOR reference — removed because it wasn’t defined
    sector = "Unknown"

    metrics_rows.append({
        'Sector': sector,
        'Stock': ticker.replace('.', '_'),
        'Model': m.upper(),
        'MSE': round(mse,2),
        'RMSE': round(rmse,2),
        'MAE': round(mae,2),
        'RMSE/MEAN': round(rmse_mean,2),
        'DirectionalAcc': round(da,2),
        'MAPE': round(mape,2)
    })

metrics_df = pd.DataFrame(
    metrics_rows,
    columns=['Sector','Stock','Model','MSE','RMSE','MAE','RMSE/MEAN','DirectionalAcc','MAPE']
)


# ------------------------------
# 9. Save CSVs
# ------------------------------
ticker_clean = ticker.replace(".", "_")
pref = ticker_clean

out_dir = "."
metrics_file = os.path.join(out_dir, f"{pref}_metrics.csv")
pred_file = os.path.join(out_dir, f"{pref}_pred.csv")
forecast_file = os.path.join(out_dir, f"{pref}_forecast5.csv")

metrics_df.to_csv(metrics_file, index=False)
all_preds_df.to_csv(pred_file, index=False)
forecast_df.to_csv(forecast_file, index=False)

# ------------------------------
# 10. Save .keras models + MODEL SUMMARY FILES
# ------------------------------
models_dir = os.path.join(out_dir, f"{pref}_models")
os.makedirs(models_dir, exist_ok=True)

for m in models:
    model_obj = retrained_models.get(m)
    if model_obj is None:
        print(f"No retrained model for {m}")
        continue

    # Save model
    model_path = os.path.join(models_dir, f"{pref}_{m}.keras")
    model_obj.save(model_path)

    # --- NEW: Print model summary ---
    print(f"\n--- {m.upper()} MODEL SUMMARY ---")
    model_obj.summary()

    # --- NEW: Save model summary to text file ---
    summary_path = os.path.join(models_dir, f"{pref}_{m}_summary.txt")
    with open(summary_path, "w") as f:
        model_obj.summary(print_fn=lambda x: f.write(x + "\n"))

    print(f"Saved model summary -> {summary_path}")

# ------------------------------
# 11. Save last 6-month PNG per model and combined PNG (Plotly)
# ------------------------------

plots_dir = os.path.join(out_dir, f"{pref}_plots")
os.makedirs(plots_dir, exist_ok=True)

# compute last-6-month threshold
if all_preds_df['Date'].dtype == 'O':
    all_preds_df['Date'] = pd.to_datetime(all_preds_df['Date'])

last_date = all_preds_df['Date'].max()
six_months_ago = last_date - pd.DateOffset(months=6)
df6 = all_preds_df[all_preds_df['Date'] >= six_months_ago].copy()

# Fallback if empty
if df6.empty and len(dates) > 0:
    six_days = last_date - pd.Timedelta(days=180)
    df_dates = pd.DataFrame({'Date': dates})
    df_tmp = pd.merge(df_dates, all_preds_df, on='Date', how='left')
    df6 = df_tmp[df_tmp['Date'] >= six_days].copy()

for m in models:
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df6['Date'], y=df6['actual'], name='Actual', mode='lines'
    ))

    if f'pred_{m}' in df6.columns:
        fig.add_trace(go.Scatter(
            x=df6['Date'], y=df6[f'pred_{m}'], 
            name=f'{m.upper()} Pred', mode='lines'
        ))

    fig.update_layout(
        title=f"{ticker_clean} - {m.upper()} Last 6 Months",
        xaxis_title="Date",
        yaxis_title="Price"
    )

    out_png = os.path.join(plots_dir, f"{pref}_{m}_6m.png")
    fig.show()
    fig.write_image(out_png, scale=2)
    print(f"Saved plot -> {out_png}")


# Combined plot (Plotly)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df6['Date'], y=df6['actual'], name='Actual', mode='lines'))

for m in models:
    if f'pred_{m}' in df6.columns:
        fig.add_trace(go.Scatter(
            x=df6['Date'], 
            y=df6[f'pred_{m}'], 
            name=f'{m.upper()} Pred', 
            mode='lines'
        ))

fig.update_layout(
    title=f"{ticker_clean} - Combined Last 6 Months",
    xaxis_title="Date",
    yaxis_title="Price"
)

combined_png = os.path.join(plots_dir, f"{pref}_ALL_6m.png")
fig.show()
fig.write_image(combined_png, scale=2)
print(f"Saved combined plot -> {combined_png}")


# ------------------------------
# 12. Directional accuracy bar chart (recompute cleanly using the aligned file)
# ------------------------------
bar_results = []
for m in models:
    pred = all_preds_df[f'pred_{m}'].dropna().values if f'pred_{m}' in all_preds_df.columns else np.array([])
    actual = all_preds_df['actual'].iloc[:len(pred)].values if len(pred) > 0 else np.array([])
    success = directional_accuracy(actual, pred) if len(pred) > 0 else np.nan
    failure = 100 - success if not np.isnan(success) else np.nan
    bar_results.append({'Model': m.upper(), 'Success': success, 'Failure': failure})

df_bar = pd.DataFrame(bar_results).set_index('Model')
fig = go.Figure(data=[
    go.Bar(name='Success', x=df_bar.index, y=df_bar['Success']),
    go.Bar(name='Failure', x=df_bar.index, y=df_bar['Failure'])
])

fig.update_layout(
    barmode='stack',
    title='Directional Accuracy (Success vs Failure) - Walk-forward',
    yaxis_title='Percentage',
    xaxis_title='Model',
    yaxis=dict(range=[0, 100])
)

bar_png = os.path.join(plots_dir, f"{pref}_directional_accuracy.png")
fig.show()
fig.write_image(bar_png, scale=2)
print(f"Saved directional accuracy plot -> {bar_png}")

print("All done.")


In [ ]:
# ------------------------------
# GPU (optional)
# ------------------------------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU(s) memory growth enabled:", gpus)
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found, using CPU")

# ------------------------------
# Config (user)
# ------------------------------

    # Auto: BAJAJ-AUTO, EICHERMOT, M&M, MARUTI, TATAMOTORS
    # Bank: AXISBANK, HDFCBANK, ICICIBANK, KOTAKBANK, SBIN
    # Chemicals: PIDILITIND, PIIND, SOLARINDS, SRF, UPL
    # FMCG: BRITANNIA, HINDUNILVR, ITC, NESTLEIND, TATACONSUM
    # Healthcare: APOLLOHOSP, CIPLA, DIVISLAB, MAXHEALTH, SUNPHARMA
    # IT: HCLTECH, INFY, TCS, TECHM, WIPRO
    # Media: NAZARA, PVRINOX, SAREGAMA, SUNTV, ZEEL
    # Metal: ADANIENT, HINDALCO, JSWSTEEL, TATASTEEL, VEDL
    # OilGas: BPCL, GAIL, IOC, ONGC, RELIANCE
    # Pharma: CIPLA, DIVISLAB, DRREDDY, LUPIN, SUNPHARMA

ticker = "BAJAJ-AUTO.NS"
end_date = datetime.today()
start_date = end_date - timedelta(days=730)
# start_date = "2023-01-01"
window = 10
step = 5
epochs = 30
batch_size = 16
forecast_days = 5
random_seed = 42
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

# ------------------------------
# 1. Load data (trading days only)
# ------------------------------
df = yf.download(ticker,
                 start=start_date.strftime('%Y-%m-%d'), #uncomment this line if using start_date = end_date - timedelta(days=730)
                 # start=start_date,
                 end=end_date.strftime('%Y-%m-%d'),
                 progress=False)

if df.empty:
    raise RuntimeError("No data downloaded - check the ticker or date range.")

# Keep only trading rows as-is (no filling); input columns: date, close
df = df[['Close']].dropna()
prices = df['Close'].values.reshape(-1, 1)   # shape (n_samples, 1)
dates = pd.to_datetime(df.index)             # DatetimeIndex

# ------------------------------
# 2. Helper: create supervised data (returns X, y)
# ------------------------------
def create_dataset(arr, window=10):
    X, y = [], []
    for i in range(len(arr) - window):
        X.append(arr[i:i+window])
        y.append(arr[i+window])
    return np.array(X), np.array(y)

# ------------------------------
# 3. Directional accuracy (expects sequences aligning to consecutive time points)
# ------------------------------
def directional_accuracy(actual, predicted):
    actual = np.array(actual).flatten()
    predicted = np.array(predicted).flatten()
    if len(actual) < 2 or len(predicted) < 2:
        return np.nan
    actual_move = actual[1:] - actual[:-1]
    pred_move = predicted[1:] - predicted[:-1]
    actual_dir = (actual_move > 0).astype(int)
    pred_dir = (pred_move > 0).astype(int)
    correct = (actual_dir == pred_dir).sum()
    total = len(actual_dir)
    return correct / total * 100  # percentage

# ------------------------------
# 4. Model factories (explicit Input to avoid warnings)
# ------------------------------
def build_lstm(window):
    inp = Input(shape=(window, 1))
    x = LSTM(64, return_sequences=True)(inp)
    x = Dropout(0.2)(x)
    x = LSTM(32)(x)
    x = Dense(16, activation='relu')(x)
    out = Dense(1)(x)
    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.Huber(delta=1))
    return model

def build_cnn(window):
    model = Sequential([
        Conv1D(32, kernel_size=2, activation='relu', padding='same', input_shape=(window,1)),
        Dropout(0.2),
        MaxPooling1D(pool_size=2, padding='same'),
        Flatten(),
        Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(1)
        ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.Huber(delta=1))
    return model

def build_hybrid(window):
    inp = Input(shape=(window,1))
    x = Conv1D(32, 2, activation='relu', padding='same')(inp)
    x = Dropout(0.2)(x)
    x = LSTM(32, kernel_regularizer=l2(0.001))(x)
    x = Dense(1)(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.Huber(delta=1))
    return model

# ------------------------------
# 5. Walk-forward evaluation (no leakage)
# ------------------------------
def walk_forward_eval(prices, dates, model_type='lstm', window=10, step=5,
                      epochs=30, batch_size=16):
    preds_all = []
    actuals_all = []
    dates_all = []

    es = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True, verbose=0)

    # sliding walk-forward: train on [0:i-1], test on [i : i+step-1]
    for i in tqdm(range(window, len(prices) - step + 1, step), desc=f"WF: {model_type}"):
        train_prices = prices[:i].copy()
        test_prices = prices[i:i+step].copy()
        train_dates = dates[:i]
        test_dates = dates[i:i+step]

        # Fit scaler on train only
        scaler = StandardScaler()
        train_scaled = scaler.fit_transform(train_prices)
        test_scaled = scaler.transform(test_prices)

        # Supervised sets
        X_train, y_train = create_dataset(train_scaled, window)
        # Prepare continuous sequence for test windows: last window points from train + test
        concat_for_test = np.vstack([train_scaled[-window:], test_scaled])
        X_test, y_test = create_dataset(concat_for_test, window)  # yields exactly `step` predictions (one per test day)

        if X_train.size == 0 or X_test.size == 0:
            continue

        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        # Build the model
        if model_type == 'lstm':
            model = build_lstm(window)
        elif model_type == 'cnn':
            model = build_cnn(window)
        else:
            model = build_hybrid(window)

        # Train on fold
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[es])

        # Predict and inverse transform using fold scaler
        pred_scaled = model.predict(X_test, verbose=0).flatten()
        pred_inv = scaler.inverse_transform(pred_scaled.reshape(-1,1)).flatten()
        actuals_inv = scaler.inverse_transform(y_test.reshape(-1,1)).flatten()

        # Append and map to test_dates (one y per test date)
        preds_all.extend(pred_inv.tolist())
        actuals_all.extend(actuals_inv.tolist())
        dates_all.extend(list(test_dates))

    if len(preds_all) == 0:
        raise ValueError("No predictions were made. Try smaller step/window or more data.")

    preds_arr = np.array(preds_all)
    actuals_arr = np.array(actuals_all)

    mse = mean_squared_error(actuals_arr, preds_arr)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actuals_arr, preds_arr)
    rmse_mean_ratio = rmse / np.mean(actuals_arr) if np.mean(actuals_arr) != 0 else np.nan
    da = directional_accuracy(actuals_arr, preds_arr)

    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'rmse_mean_ratio': rmse_mean_ratio,
        'directional_acc': da,
        'preds': preds_arr,
        'actuals': actuals_arr,
        'dates': pd.to_datetime(dates_all)
    }

# ------------------------------
# 6. Run walk-forward for each model and collect results
# ------------------------------
models = ['cnn', 'lstm', 'hybrid']
results = {}
all_preds_df = None

for m in models:
    res = walk_forward_eval(prices, dates, model_type=m, window=window, step=step,
                            epochs=epochs, batch_size=batch_size)
    results[m] = {
        'MSE': res['mse'],
        'RMSE': res['rmse'],
        'MAE': res['mae'],
        'RMSE/MEAN': res['rmse_mean_ratio'],
        'DirectionalAcc': res['directional_acc']
    }
    print(f"\n{m.upper()} RESULTS (de-scaled):")
    print(f"  MSE: {res['mse']:.2f}, RMSE: {res['rmse']:.2f}, MAE: {res['mae']:.2f}, RMSE/MEAN: {res['rmse_mean_ratio']:.2f}, DirectionalAcc: {res['directional_acc']:.2f}%")

    # Build df for model predictions aligned by date
    df_m = pd.DataFrame({
        'Date': res['dates'],
        f'actual_{m}': res['actuals'],
        f'pred_{m}': res['preds']
    })
    if all_preds_df is None:
        all_preds_df = df_m
    else:
        all_preds_df = pd.merge(all_preds_df, df_m, on='Date', how='outer')

# Sort and clean aggregated predictions
all_preds_df = all_preds_df.sort_values('Date').reset_index(drop=True)

# Consolidate actual columns (they should be identical across models)
actual_cols = [c for c in all_preds_df.columns if c.startswith('actual_')]
if len(actual_cols) > 0:
    all_preds_df['actual'] = all_preds_df[actual_cols].bfill(axis=1).iloc[:, 0]
    all_preds_df.drop(columns=actual_cols, inplace=True)

# Reorder columns
cols = ['Date', 'actual'] + [c for c in all_preds_df.columns if c.startswith('pred_')]
all_preds_df = all_preds_df[cols]

# ------------------------------
# 7. Final retrain on full history for forecast
# ------------------------------
def retrain_and_forecast(prices, dates, model_type='lstm', window=10, forecast_days=5,
                         epochs=30, batch_size=16):
    # Fit scaler on full past history (allowed for forecasting)
    scaler_full = StandardScaler()
    scaled_full = scaler_full.fit_transform(prices)

    X_full, y_full = create_dataset(scaled_full, window)
    X_full = X_full.reshape((X_full.shape[0], X_full.shape[1], 1))

    if model_type == 'lstm':
        model = build_lstm(window)
    elif model_type == 'cnn':
        model = build_cnn(window)
    else:
        model = build_hybrid(window)

    es = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True, verbose=0)
    model.fit(X_full, y_full, epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[es])

    # Recursive forecast on business days (next trading days)
    last_window = scaled_full[-window:].reshape(1, window, 1)
    preds_scaled = []
    temp = last_window.copy()
    for _ in range(forecast_days):
        f = model.predict(temp, verbose=0)
        preds_scaled.append(float(f.flatten()[0]))
        f_reshaped = f.reshape((1,1,1))
        temp = np.append(temp[:,1:,:], f_reshaped, axis=1)

    preds = scaler_full.inverse_transform(np.array(preds_scaled).reshape(-1,1)).flatten()
    future_dates = pd.date_range(start=dates[-1], periods=forecast_days+1, freq='B')[1:]
    return pd.DataFrame({'Date': future_dates, f'{model_type}_forecast': preds}), model

# Generate forecast_df with business days (and collect last retrained models)
forecast_dates = pd.date_range(start=dates[-1], periods=forecast_days+1, freq='B')[1:]
forecast_df = pd.DataFrame({'Date': forecast_dates})
retrained_models = {}
for m in models:
    fc_df, retrained_model = retrain_and_forecast(prices, dates, model_type=m, window=window, forecast_days=forecast_days,
                                                  epochs=epochs, batch_size=batch_size)
    forecast_df = pd.merge(forecast_df, fc_df, on='Date', how='left')
    retrained_models[m] = retrained_model  # store the retrained model (trained on full history)

# ------------------------------
# 8. Build metrics dataframe (with requested columns)
# ------------------------------
# compute MAPE helper (safe)
def compute_mape(a, p):
    a = np.array(a).astype(float)
    p = np.array(p).astype(float)
    mask = a != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((a[mask] - p[mask]) / a[mask])) * 100

metrics_rows = []
for m in models:
    preds = all_preds_df[f'pred_{m}'].dropna().values
    # align actuals length-wise
    actual = all_preds_df['actual'].iloc[:len(preds)].values if len(preds) > 0 else np.array([])
    if len(preds) == 0:
        mse = rmse = mae = rmse_mean = np.nan
        da = np.nan
        mape = np.nan
    else:
        mse = mean_squared_error(actual, preds)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actual, preds)
        rmse_mean = rmse / np.mean(actual) if np.mean(actual) != 0 else np.nan
        da = directional_accuracy(actual, preds)
        mape = compute_mape(actual, preds)

    base = ticker.split('.')[0]  # remove .NS
    sector = STOCK_TO_SECTOR.get(base, "Unknown")
    
    metrics_rows.append({
        'Sector': sector,                      # unknown, leave empty
        'Stock': ticker.replace('.', '_'),
        'Model': m.upper(),
        'MSE': round(mse,2),
        'RMSE': round(rmse,2),
        'MAE': round(mae,2),
        'RMSE/MEAN': round(rmse_mean,2),
        'DirectionalAcc': round(da,2),
        'MAPE': round(mape,2)
    })

metrics_df = pd.DataFrame(metrics_rows, columns=['Sector','Stock','Model','MSE','RMSE','MAE','RMSE/MEAN','DirectionalAcc','MAPE'])

# ------------------------------
# Build output folder structure
# ------------------------------

# Clean ticker for folder name (RELIANCE.NS → RELIANCE)
ticker_clean = ticker.split(".")[0]

# Determine sectors for this stock
sectors = STOCK_TO_SECTOR.get(ticker_clean, ["Unknown"])

# Final paths will be stored here
save_paths = []

for sector in sectors:
    base_path = os.path.join("output", "stock_data", sector, ticker_clean)
    os.makedirs(base_path, exist_ok=True)
    save_paths.append(base_path)


# ------------------------------
# 9. Save CSVs with short meaningful names (ticker first)
# ------------------------------
ticker_clean = ticker.replace(".", "_")
pref = ticker_clean

out_dir = "."  # current working directory in Kaggle
metrics_file = os.path.join(out_dir, f"{pref}_metrics.csv")
pred_file = os.path.join(out_dir, f"{pref}_pred.csv")
forecast_file = os.path.join(out_dir, f"{pref}_forecast5.csv")

# ------------------------------
# 9. Save CSVs into each sector folder
# ------------------------------
for path in save_paths:
    metrics_df.to_csv(os.path.join(path, f"{ticker_clean}_metrics.csv"), index=False)
    all_preds_df.to_csv(os.path.join(path, f"{ticker_clean}_pred.csv"), index=False)
    forecast_df.to_csv(os.path.join(path, f"{ticker_clean}_forecast5.csv"), index=False)


print(f"Saved metrics CSV -> {metrics_file}")
print(f"Saved actual vs pred CSV -> {pred_file}")
print(f"Saved forecast CSV -> {forecast_file}")

# ------------------------------
# 10. Save models + summaries
# ------------------------------
for path in save_paths:
    models_dir = os.path.join(path, "models")
    os.makedirs(models_dir, exist_ok=True)

    for m in models:
        model_obj = retrained_models[m]
        model_file = os.path.join(models_dir, f"{ticker_clean}_{m}.keras")
        summary_file = os.path.join(models_dir, f"{ticker_clean}_{m}_summary.txt")

        model_obj.save(model_file)
        with open(summary_file, "w") as f:
            model_obj.summary(print_fn=lambda x: f.write(x + "\n"))


# ==========================================================
# 11 & 12. SAVE PLOTS (Last 6M + Combined + Directional Acc)
# ==========================================================

# -------------- Folder structure (NEW LOGIC) --------------
ticker_clean = ticker.split(".")[0]       # "RELIANCE"
sectors = STOCK_TO_SECTOR.get(ticker_clean, ["Unknown"])

# sectors must always be a list
if isinstance(sectors, str):
    sectors = [sectors]

# Build per-sector output folders (Option A)
sector_plot_dirs = []
for sec in sectors:
    sec_path = os.path.join(
        "output", "stock_data", sec, ticker_clean, "plots"
    )
    os.makedirs(sec_path, exist_ok=True)
    sector_plot_dirs.append(sec_path)

# ----------------- Create last-6-months df ----------------
if all_preds_df['Date'].dtype == 'O':
    all_preds_df['Date'] = pd.to_datetime(all_preds_df['Date'])

last_date = all_preds_df['Date'].max()
six_months_ago = last_date - pd.DateOffset(months=6)
df6 = all_preds_df[all_preds_df['Date'] >= six_months_ago].copy()

# fallback if needed
if df6.empty and len(dates) > 0:
    six_days = last_date - pd.Timedelta(days=180)
    df_dates = pd.DataFrame({'Date': dates})
    df_tmp = pd.merge(df_dates, all_preds_df, on='Date', how='left')
    df6 = df_tmp[df_tmp['Date'] >= six_days].copy()

# ==========================================================
# 11A. SAVE Last 6-month PLOTS for each model
# ==========================================================
for m in models:
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df6['Date'], y=df6['actual'], name="Actual", mode="lines"
    ))

    pred_col = f"pred_{m}"
    if pred_col in df6.columns:
        fig.add_trace(go.Scatter(
            x=df6['Date'], y=df6[pred_col], 
            name=f"{m.upper()} Pred", mode="lines"
        ))

    fig.update_layout(
        title=f"{ticker_clean} - {m.upper()} Last 6 Months",
        xaxis_title="Date",
        yaxis_title="Price"
    )

    # Save to ALL sector folders
    for sec_dir in sector_plot_dirs:
        out_png = os.path.join(sec_dir, f"{ticker_clean}_{m}_6m.png")
        fig.write_image(out_png, scale=2)
        print(f"Saved -> {out_png}")

# ==========================================================
# 11B. Combined 6-month PLOT
# ==========================================================
fig = go.Figure()
fig.add_trace(go.Scatter(x=df6['Date'], y=df6['actual'], name="Actual", mode="lines"))

for m in models:
    pred_col = f"pred_{m}"
    if pred_col in df6.columns:
        fig.add_trace(go.Scatter(
            x=df6['Date'], y=df6[pred_col], 
            name=f"{m.upper()} Pred", mode="lines"
        ))

fig.update_layout(
    title=f"{ticker_clean} - Combined Last 6 Months",
    xaxis_title="Date",
    yaxis_title="Price"
)

# Save combined plot in all sector folders
for sec_dir in sector_plot_dirs:
    combined_png = os.path.join(sec_dir, f"{ticker_clean}_ALL_6m.png")
    fig.write_image(combined_png, scale=2)
    print(f"Saved -> {combined_png}")

# ==========================================================
# 12. Directional Accuracy Bar Chart
# ==========================================================
bar_results = []
for m in models:
    pred_col = f"pred_{m}"
    preds = df6[pred_col].dropna().values if pred_col in df6.columns else np.array([])
    actual = df6['actual'].iloc[:len(preds)].values if len(preds) > 0 else np.array([])

    da = directional_accuracy(actual, preds) if len(preds) > 0 else np.nan
    fail = 100 - da if not np.isnan(da) else np.nan

    bar_results.append({'Model': m.upper(), 'Success': da, 'Failure': fail})

df_bar = pd.DataFrame(bar_results).set_index("Model")

fig = go.Figure(data=[
    go.Bar(name="Success", x=df_bar.index, y=df_bar["Success"]),
    go.Bar(name="Failure", x=df_bar.index, y=df_bar["Failure"])
])

fig.update_layout(
    barmode="stack",
    title="Directional Accuracy (Success vs Failure) - Walk-forward",
    yaxis_title="Percentage",
    xaxis_title="Model",
    yaxis=dict(range=[0, 100])
)

# Save bar chart to each sector folder
for sec_dir in sector_plot_dirs:
    bar_png = os.path.join(sec_dir, f"{ticker_clean}_directional_accuracy.png")
    fig.write_image(bar_png, scale=2)
    print(f"Saved -> {bar_png}")

print("All plots saved.")


print("All done.")


GPU(s) memory growth enabled: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


WF: cnn:   0%|          | 0/140 [00:00<?, ?it/s]I0000 00:00:1763460862.005729      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763460862.006488      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
I0000 00:00:1763460865.001620     127 service.cc:148] XLA service 0x7b6b5c004ca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763460865.003271     127 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1763460865.003290     127 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1763460865.327005     127 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1763460867.7709

WF: cnn:   4%|▍         | 6/140 [00:23<09:15,  4.15s/it]

WF: cnn: 100%|██████████| 140/140 [11:58<00:00,  5.13s/it]



CNN RESULTS (de-scaled):
  MSE: 115803.705426, RMSE: 340.299435, MAE: 246.325180, RMSE/MEAN: 0.046227, DirectionalAcc: 51.01%


...

Saved plot -> ./BAJAJ-AUTO_NS_plots/BAJAJ-AUTO_NS_cnn_6m.png


Saved plot -> ./BAJAJ-AUTO_NS_plots/BAJAJ-AUTO_NS_lstm_6m.png


Saved plot -> ./BAJAJ-AUTO_NS_plots/BAJAJ-AUTO_NS_hybrid_6m.png


Saved combined plot -> ./BAJAJ-AUTO_NS_plots/BAJAJ-AUTO_NS_ALL_6m.png


Saved directional accuracy plot -> ./BAJAJ-AUTO_NS_plots/BAJAJ-AUTO_NS_directional_accuracy.png
All done.
